In [ ]:
import pandas as pd
import numpy as np
import math

In [ ]:
#define class and node
class node(Object): # object is required
    def __init__(self, val): # self is required
        self.val = val
        self.child = None  # child should be a node series
    def __str__(self):
        return str(self.val)
# decision tree
class Tree(Object):
    def __init__(self, val): # value = label
        self.root = node(val)
    def add(self, child:node): # child is a node
        item = node(child)
        if not self.root:
            self.root = item # if there is no nodes in tree at all
        else:
            self.child.append(child)

    def addchild(self, children:listofnode):        
        for child_item in children:
            self.add(child_item)
    
    def getchild(self):
        return self.child

In [1]:
def calIG(data, fea):
    #this function is designed to canculate the information gain of a specific feature to the dataset, dataset: sample,feature， fea: index of the feature
    #calculate the empirical entropy of dataset d
    sample = data.shape[0]
    groups = data.groupby(df.iloc[-1])
    hd = -sum(groups.count()/sample*np.log2(groups.count()/sample))
    #calculate the coditional entropy hda
    hda = -sum([sum(group.groupby(df.iloc[fea]).count()/group.count()*np.log2(group.groupby(df.iloc[fea]).count()/group.count())) for group in groups])
    #calculate information gain
    ig = hd - hda
    return ig

In [ ]:
def calIGR(data, fea):
    #calculate the information gain ratio
    ig = calIG(data, fea)
    groups = data.groupby(df.iloc[fea])
    deno = -sum(groups.count()*np.log2(groups.count()))
    igr = ig/deno
    return igr

In [ ]:
def id3(data, features, epsilon):
    # this is the id3 algorithm for decision tree
    # case 1: if all samples belong to one class, that is ,there is only one label, return a single-node tree which root is the only label ck
    labels = np.unique(data.iloc[-1])
    if len(labels)==1:
        return Tree(lables[0])
    # case 2: if there is no feature, return a single-node tree which root is the max ck
    if len(features)==0:
        return Tree(max(data.iloc[-1]))
    # case 3: normal case
    pass